<hr size="5" />

### **<font color='DarkCyan'>Salzburg University of Applied Sciences - 2023**  
#### **<font color='DarkCyan'>Information Technology & Systems Engineering**

# **<font color='GoldenRod'>Master Thesis**  
## **<font color='GoldenRod'>Deep Learning for Advancing Animal Breeding: A Study on Austrian Fleckvieh Cattle**

<hr size="5">

#### Student Name: Jakob Ganitzer
#### Degree Program:  ITSM-B

<hr size="5" />

### Imports

In [ ]:
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import torch
from torch_geometric.data import Dataset, Data


### Load Data

In [2]:
dataRoot = "/data"

In [3]:
heritability = "/h40"

In [4]:
phenotype_path = dataRoot + heritability + heritability + "_simu.dat"
snp_path = dataRoot + heritability + heritability + "_simu.snp"
benchmark_path = dataRoot + heritability + heritability + "_simu.bv"
pedigree_path = dataRoot + heritability + heritability + "_simu.ped"
snpPos_path = dataRoot + heritability + heritability + "_simu_snp.txt"
qtl_path = dataRoot + heritability + heritability + "_simu_qtl.txt"

In [6]:
data = torch.load(dataRoot + heritability + "/_.pt")

In [13]:
df_ped = pd.read_csv(pedigree_path, sep=" ", header=None, names=["id", "sire", "dam", "generation", "sex"])

In [23]:
df_snp_gen30_ped = pd.read_parquet(dataRoot + heritability + '/snpTruePheno_gen30_BPpos_maf_one_hot_ped.parquet', engine='pyarrow')

In [5]:
data = torch.load(dataRoot + heritability + "/graph_maf.pt")

In [47]:
df_pheno = pd.read_csv(phenotype_path , sep=" ", header=None, names=["id", "fixed_effect", "phenotype"])

In [8]:
df_pheno.head()

,id,fixed_effect,phenotype
0,201.0,1,0.766089
1,202.0,1,1.853758
2,203.0,1,0.970537
3,204.0,1,0.362347
4,205.0,1,0.875165


In [9]:
# load data
#data = torch.load("data_v1.pt")

In [8]:
df_ids = df_snp_pheno_ped[["id"]]

# drop id and phenotype columns
df_snp_temp = df_snp_pheno_ped.drop(["id", "phenotype", "sire", "dam", "generation", "sex"], axis=1)

In [11]:
# Convert the data frame back to a numpy array
snp_array = df_snp_temp.to_numpy()

# Reshape the data back to its original shape
snp_one_hot = snp_array.reshape(df_snp_temp.shape[0], int(df_snp_temp.shape[1]/3), 3)

## Graph Generation

In [61]:
df_snp_pheno_ped = pd.read_parquet(dataRoot + heritability + '/snp_maf_one_hot_ped_gen30.parquet', engine='pyarrow')

In [6]:
df_snp_pheno_ped = pd.read_parquet(dataRoot + heritability + '/snp_maf_one_hot_ped.parquet', engine='pyarrow')

In [7]:
df_snp_sire = pd.read_parquet(dataRoot + heritability + '/snp_sire_BPpos_maf_one_hot_gen30.parquet', engine='pyarrow')

In [ ]:
df_snp_sire = pd.read_parquet(dataRoot + heritability + '/snp_sire_BPpos_maf_gen30.parquet', engine='pyarrow')

In [19]:
df_pheno = pd.read_csv(phenotype_path , sep=" ", header=None, names=["id", "fixed_effect", "phenotype"])

In [20]:
df_pheno

,id,fixed_effect,phenotype
0,201.0,1,0.766089
1,202.0,1,1.853758
2,203.0,1,0.970537
3,204.0,1,0.362347
4,205.0,1,0.875165
...,...,...,...
650995,1260192.0,1,14.630073
650996,1260194.0,1,11.466613
650997,1260196.0,1,13.342062
650998,1260198.0,1,11.703152


In [21]:
df_pheno.drop(["fixed_effect"], axis=1, inplace = True)

In [24]:
true_pheno = df_snp_gen30_ped[['id', 'phenotype']].copy()

In [25]:
df_pheno = pd.concat([df_pheno, true_pheno], ignore_index=True)

In [28]:
df_ids = df_snp_pheno_ped[["id"]]
df_snp_temp = df_snp_pheno_ped.drop(["id", "phenotype", "sire", "dam", "generation", "sex"], axis=1)

In [111]:
df_sire_ids = df_snp_sire[["id"]]
df_snp_sire_temp = df_snp_sire.drop(["id"], axis=1)

In [112]:
result_df = pd.concat([df_snp_temp, df_snp_sire_temp], ignore_index=True)

In [113]:
result_id_df = pd.concat([df_ids, df_sire_ids], ignore_index=True)

In [ ]:
snp_array = result_df.to_numpy()

In [30]:
#ordinary
def add_value(x, value):
    return x + value

df_snp_temp = df_snp_temp.apply(add_value, value=7)

snp_array = df_snp_temp.to_numpy()

In [25]:
# Reshape the data back to its original shape
snp_one_hot = snp_array.reshape(df_snp_temp.shape[0], int(df_snp_temp.shape[1]/3), 3)

In [115]:
snp_one_hot = snp_array.reshape(result_df.shape[0], int(result_df.shape[1]/3), 3)

In [38]:
animal_ids = df_snp_pheno_ped['id'].unique()
dam_ids = df_snp_pheno_ped['dam'].unique()
sire_ids = df_snp_pheno_ped['sire'].unique()

In [39]:
id_to_index = {id: index for index, id in enumerate({*animal_ids, *dam_ids, *sire_ids})}
#id_to_index = {id: index for index, id in enumerate({*animal_ids, *dam_ids})}

In [40]:
index_to_id = {index: id for id, index in id_to_index.items()}

In [41]:
#ordinary encoding
# dictinary of id to one hot encoded snp sequence
id_to_snp = dict(zip(df_ids["id"], torch.from_numpy(snp_array)))

# replace the id ind id_to_snp_one_hot with the corresponding index in id_to_index
index_to_snp = {id_to_index[id]: snp for id, snp in id_to_snp.items()}


In [30]:
# dictinary of id to one hot encoded snp sequence
id_to_snp_one_hot = dict(zip(df_ids["id"], torch.from_numpy(snp_one_hot)))

# replace the id ind id_to_snp_one_hot with the corresponding index in id_to_index
index_to_snp_one_hot = {id_to_index[id]: snp for id, snp in id_to_snp_one_hot.items()}

In [120]:
# with sire genotype
# dictinary of id to one hot encoded snp sequence
id_to_snp_one_hot = dict(zip(result_id_df["id"], torch.from_numpy(snp_one_hot)))

# replace the id ind id_to_snp_one_hot with the corresponding index in id_to_index
index_to_snp_one_hot = {id_to_index[id]: snp for id, snp in id_to_snp_one_hot.items()}

In [42]:
#ordinary
num_features = snp_array.shape[1]
node_features = torch.zeros(len(id_to_index), num_features) # ordinal encoding
node_labels = torch.tensor(np.full(len(id_to_index), np.nan), dtype=torch.float32)

In [43]:
#ordinary
# replace the values in node_features whith the values from index_to_snp_one_hot by the corresponding index
for index, snp in index_to_snp.items():
    node_features[index] = snp

In [44]:
# Convert the tensor to integer data type
node_features = node_features.to(torch.int)

In [46]:
node_features.shape

torch.Size([284992, 26282])

In [19]:
#num_features = len([col for col in df_snp_pheno_ped.columns if col.startswith('snp_')])
num_features = snp_one_hot.shape[1]

#node_features = torch.zeros(len(id_to_index), num_features) # ordinal encoding
node_features2 = torch.zeros(len(id_to_index), num_features, 3) # one-hot encoding
node_labels = torch.zeros(len(id_to_index))

In [121]:
#num_features = len([col for col in df_snp_pheno_ped.columns if col.startswith('snp_')])
num_features = snp_one_hot.shape[1]

#node_features = torch.tensor(np.full((len(id_to_index), num_features), np.nan), dtype=torch.float16) # ordinally encoded
#node_features = torch.tensor(np.full((len(id_to_index), num_features, 3), np.nan), dtype=torch.float16) # one-hot encoded
node_features = torch.zeros(len(id_to_index), num_features, 3) # one-hot encoding

node_labels = torch.tensor(np.full(len(id_to_index), np.nan), dtype=torch.float32)

In [122]:
# replace the values in node_features whith the values from index_to_snp_one_hot by the corresponding index
for index, snp in index_to_snp_one_hot.items():
    node_features[index] = snp

In [47]:
# get indices from id_to_index with animal_ids and convert to tensor
animal_indices = torch.tensor([id_to_index[id] for id in animal_ids])

In [48]:
# Create edge indices
valid_dam_indices = df_snp_pheno_ped['dam'].map(id_to_index).notna()
valid_sire_indices = df_snp_pheno_ped['sire'].map(id_to_index).notna()
#animal_indices = animal_indices[valid_animal_indices]

dam_indices = torch.tensor(df_snp_pheno_ped.loc[valid_dam_indices, 'dam'].map(id_to_index))
sire_indices = torch.tensor(df_snp_pheno_ped.loc[valid_sire_indices, 'sire'].map(id_to_index))

appended_animal_indices = torch.cat((animal_indices, animal_indices))
appended_parent_indices = torch.cat((dam_indices, sire_indices))

edge_indices = torch.cat((appended_parent_indices, appended_animal_indices)).view(2, len(appended_animal_indices))

#edge_indices = torch.cat((animal_indices, dam_indices)).view(2, len(animal_indices))


In [52]:
data = Data(x=node_features, edge_index=edge_indices, y=node_labels)

In [53]:
data

Data(x=[284992, 26282], edge_index=[2, 504000], y=[284992])

In [54]:
len(node_labels)

284992

In [55]:
data.node_ids = torch.tensor([index_to_id[index] for index in range(len(node_labels))])


In [57]:
id_pheno_dict = df_pheno.set_index('id')['phenotype'].astype(float).to_dict()

In [58]:
# Assing label to dam nodes without features but labels

id_pheno_dict = df_pheno.set_index('id')['phenotype'].astype(float).to_dict()

labels = [id_pheno_dict.get(node_id.item(), float('nan')) for node_id in data.node_ids]
labels = torch.tensor(labels, dtype=torch.float32)

data.y = labels

In [60]:
y_has_nans = torch.isnan(data.y)

In [61]:
y_has_nans.sum()

tensor(2000)

In [62]:
data.x.shape[1]

26282

In [63]:
data.y

tensor([ 9.7124, 11.3992,  9.3959,  ..., 12.1882,  8.5984,  9.7769])

In [92]:
torch.save(data, dataRoot + heritability + "/graph_maf.pt")

In [ ]:
# load data
data = torch.load(dataRoot + heritability + "/graph_maf.pt")

In [67]:
import random
import torch

def split_data(_data, val_size=0.2, test_size=0.1, random_seed=42):
    # Set the random seed for reproducibility
    torch.manual_seed(random_seed)

    # Check if data.y contains NaN values
    y_has_nans = torch.isnan(_data.y)
    
    print(type(y_has_nans))
    
    # Check if data.x has only zeros
    x_has_only_zeros = torch.sum(_data.x, dim=(1, 2)) == 0

    # Combine the conditions using logical OR
    has_nans = y_has_nans | x_has_only_zeros

    # Negate the boolean tensor to get the desired result
    has_no_nans = ~has_nans

    # Identify the indices where 'has_no_nans' is True
    true_indices = torch.where(has_no_nans)[0]

    # Determine the number of indices to change
    num_indices = len(true_indices)
    num_test_indices = int(num_indices * test_size)
    num_val_indices = int(num_indices * val_size)

    # Generate random indices for test, validation, and train sets
    indices = torch.randperm(num_indices)
    test_indices = true_indices[indices[:num_test_indices]]
    val_indices = true_indices[indices[num_test_indices:num_test_indices + num_val_indices]]
    train_indices = true_indices[indices[num_test_indices + num_val_indices:]]

    # Create the masks
    train_mask = torch.zeros_like(has_no_nans, dtype=torch.bool)
    val_mask = torch.zeros_like(has_no_nans, dtype=torch.bool)
    test_mask = torch.zeros_like(has_no_nans, dtype=torch.bool)

    train_mask[train_indices] = True
    val_mask[val_indices] = True
    test_mask[test_indices] = True

    return train_mask, val_mask, test_mask


def split_data_Gen30(_data, val_size=0.2, test_size=0.1, random_seed=42):
    # Set the random seed for reproducibility
    torch.manual_seed(random_seed)

    # Check if data.y contains NaN values
    y_has_nans = torch.isnan(_data.y)
    
    # Check if data.x has only zeros
    x_has_only_zeros = torch.sum(_data.x, dim=(1, 2)) == 0
    #x_has_only_zeros = torch.sum(_data.x, dim=1) == 0 # ordinary
    
    # Convert the 'id' column of the DataFrame to a Python set for faster lookup
    snp_ids_set = set(df_snp_gen30_ped['id'])

    # Convert the tensor to a Python list or numpy array to iterate through
    node_ids_list = _data.node_ids.tolist()

    # Create a list of boolean values indicating if an ID is in the snp_ids_set
    bool_list = [id_item in snp_ids_set for id_item in node_ids_list]

    # Convert the list to a boolean tensor
    bool_tensor_gen30 = torch.tensor(bool_list, dtype=torch.bool)
    
    
    # Combine the conditions using logical OR
    has_nans = y_has_nans | x_has_only_zeros | bool_tensor_gen30

    # Negate the boolean tensor to get the desired result
    has_no_nans = ~has_nans

    # Identify the indices where 'has_no_nans' is True
    true_indices = torch.where(has_no_nans)[0]

    # Determine the number of indices to change
    num_indices = len(true_indices)
    num_test_indices = int(num_indices * test_size)
    num_val_indices = int(num_indices * val_size)

    # Generate random indices for test, validation, and train sets
    indices = torch.randperm(num_indices)
    test_indices = true_indices[indices[:num_test_indices]]
    val_indices = true_indices[indices[num_test_indices:num_test_indices + num_val_indices]]
    train_indices = true_indices[indices[num_test_indices + num_val_indices:]]

    # Create the masks
    train_mask = torch.zeros_like(has_no_nans, dtype=torch.bool)
    val_mask = torch.zeros_like(has_no_nans, dtype=torch.bool)
    test_mask = torch.zeros_like(has_no_nans, dtype=torch.bool)

    train_mask[train_indices] = True
    val_mask[val_indices] = True
    test_mask[test_indices] = True

    return train_mask, val_mask, test_mask, bool_tensor_gen30, has_no_nans

#train_mask, val_mask, test_mask = split_data(data, val_size=0.2, test_size=0.1, random_seed=42)
train_mask, val_mask, test_mask, gen30_mask, fineTrain_mask = split_data_Gen30(data, val_size=0.1, test_size=0.1, random_seed=42)

In [68]:
train_mask.sum()

tensor(168000)

In [69]:
test_mask.sum()

tensor(21000)

In [70]:
val_mask.sum()

tensor(21000)

In [71]:
fineTrain_mask.sum()

tensor(210000)

In [72]:
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
data.gen30_mask = gen30_mask
data.fineTrain_mask = fineTrain_mask

In [75]:
data

Data(x=[284992, 26282], edge_index=[2, 504000], y=[284992], node_ids=[284992], train_mask=[284992], val_mask=[284992], test_mask=[284992], gen30_mask=[284992], fineTrain_mask=[284992])

In [140]:
# Create a boolean mask
gen30_mask = data.gen30_mask

# Apply the mask to data.y
new_tensor = data.y[gen30_mask]

print(new_tensor)

tensor([11.5899, 12.9473, 13.4185,  ..., 12.8304, 12.4708, 12.8761])


In [145]:
# Convert data_node_ids to a numpy array and then to a list
node_ids_list = data.node_ids.numpy().astype(int).tolist()

# Filter node_ids based on id column in df_snp_sire
filtered_indices = [index for index, node_id in enumerate(node_ids_list) if node_id in df_snp_sire['id'].tolist()]

# Create a new tensor using the filtered indices
new_tensor = data.y[torch.tensor(filtered_indices)]

torch.Size([1800])


In [20]:
import torch.nn.functional as F

def create_attention_mask(data, paddedLength=-1):

    # Create attention_mask (torch.FloatTensor of shape (batch_size, sequence_length)) for train and val
    attention_mask = torch.ones(data.x.shape[0], data.x.shape[1]).to('cpu')

    if(paddedLength != -1 and paddedLength > data.x.shape[1]):
        pad = paddedLength - data.x.shape[1]
        # Pad the data to the max length
        X_pad = np.pad(data.x.to('cpu'), ((0,0),(0,pad),(0,0)), 'constant', constant_values=0)
        attention_mask = F.pad(attention_mask, (0,pad), 'constant', 0)

    data.x_pad = torch.tensor(X_pad, dtype=torch.float16)
    data.attention_mask = attention_mask

In [87]:
import torch.nn.functional as F

def create_attention_mask_cat(data, paddedLength=-1):

    # Create attention_mask (torch.FloatTensor of shape (batch_size, sequence_length)) for train and val
    attention_mask = torch.ones(data.x.shape[0], data.x.shape[1]).to('cpu')

    if(paddedLength != -1 and paddedLength > data.x.shape[1]):
        pad = paddedLength - data.x.shape[1]
        # Pad the data to the max length
        X_pad = np.pad(data.x.to('cpu'), ((0,0),(0,pad)), 'constant', constant_values=4)
        attention_mask = F.pad(attention_mask, (0,pad), 'constant', 0)

    data.x_pad = torch.tensor(X_pad, dtype=torch.int8)
    data.attention_mask = attention_mask

In [22]:
create_attention_mask(data, paddedLength=26624)

In [88]:
create_attention_mask_cat(data, paddedLength=26624)

In [31]:
data.validate(raise_on_error=True)

True

In [32]:
print(data.keys)

['node_ids', 'edge_index', 'y', 'test_mask', 'x_pad', 'val_mask', 'attention_mask', 'x', 'train_mask']


In [6]:
for key, item in data:
    print(f'{key} found in data')

x found in data
edge_index found in data
y found in data
node_ids found in data
train_mask found in data
val_mask found in data
test_mask found in data


In [8]:
data.num_nodes

240992

In [9]:
data.num_edges

210000

In [10]:
data.num_node_features

3

In [11]:
data.has_isolated_nodes()

False

In [12]:
data.has_self_loops()

False

In [13]:
data.is_directed()

True

In [86]:
import torch
from torch_geometric.utils import contains_self_loops

# Check if the graph contains self-loops
has_self_loops = contains_self_loops(data.edge_index)

if has_self_loops:
    # Find the indices of nodes with self-loops
    self_loop_indices = (data.edge_index[0] == data.edge_index[1]).nonzero().view(-1)
    nodes_with_self_loops = self_loop_indices.unique()

    print("Nodes with self-loops:")
    print(nodes_with_self_loops)
else:
    print("The graph does not contain any self-loops.")


The graph does not contain any self-loops.


In [43]:
# get the highest node index
highest_node_index = data.node_ids.max().item()
highest_node_index

1260200.0

In [ ]:
# Convert edge_index to NetworkX graph
import networkx as nx

graph = nx.DiGraph()
source, target = data.edge_index
graph.add_edges_from(zip(source.tolist(), target.tolist()))

# Plot the graph
pos = nx.fruchterman_reingold_layout(graph) 
plt.figure(figsize=(20, 20))
labels = {node: label for node, label in enumerate(data.node_ids.tolist())}
nx.draw(graph, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=8, labels=labels)
plt.show()

In [68]:
nan_indices = torch.isnan(data.y)
nan_node_ids = torch.nonzero(nan_indices).squeeze()

print("Node IDs with NaN labels:")
print(len(nan_node_ids))


Node IDs with NaN labels:
44000


In [45]:
#nan_indices = torch.isnan(data.x).any(dim=1)
nan_indices =  torch.isnan(data.x).any(dim=2).any(dim=1)
nan_node_ids = torch.nonzero(nan_indices).squeeze()

print("Node IDs with NaN node features:")
print(len(nan_node_ids))


Node IDs with NaN node features:
0
